In [7]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
from mylibs import *
import os

In [8]:
def get_data(url:str) -> pd.DataFrame:
    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')
    product_container = soup.find(name = 'div', attrs =  {'class' :'block-card-cate container-card-cate d-flex justify-content-flex-start flex-wrap w-10'})
    
    list_product_names = []
    tag_product_names = product_container.find_all(name= 'h3')
    for tag_product_name in tag_product_names:
        product_name = tag_product_name.text
        list_product_names.append(product_name)
        
    list_product_prices = []
    tag_product_prices = product_container.find_all(name= 'div', attrs = {'class' : 'price-card primary-price'})
    for tag_product_price in tag_product_prices:
        product_price = tag_product_price.text.split('đ')[0].replace('.', '').replace('                        ', '')
        list_product_prices.append(product_price)
        
    list_image_links = []
    tag_image_links = product_container.find_all(name= 'div', attrs = {'class' : 'image-card-cate'})
    for tag_image_link in tag_image_links:
        image_link = tag_image_link.find('img')['data-src']
        list_image_links.append(image_link)
        # img_name = image_link.split('/')[-1]
        # download_image(image_link, f'tivi/{img_name}')
    secondary_list = []
    tag_secondary_prices = product_container.find_all(name= 'div', attrs = {'class' : 'price-card secondary-price'})
    for tag_secondary_price in tag_secondary_prices:
        # print(tag_secondary_price)
        if(tag_secondary_price is None):
            secondary_list.append(0)
            continue
        secondary_price = int(tag_secondary_price.find(name = 'span').text.replace(',', '').split('đ')[0].strip())
       
        secondary_list.append(secondary_price)

            
    print(len(list_product_names), len(list_product_prices), len(list_image_links), len(secondary_list))
    df = pd.DataFrame({'product_name': list_product_names, 'product_price': list_product_prices, 'second_product_price' : secondary_list , 'image_link': list_image_links})
    return df        

In [9]:
list_links = []
for page in range(1,15):
    page_link = f'https://dienmaythiennamhoa.vn/tivi.html?page={page}&_is_desktop=1&_is_robot=0&sort='
    page_response = requests.get(page_link)
    page_content = BeautifulSoup(page_response.content, 'html.parser')
    
    product_container = page_content.find(name = 'div', attrs =  {'class' :'block-card-cate container-card-cate d-flex justify-content-flex-start flex-wrap w-10'}) 
    tag_links = product_container.find_all(name = 'a', attrs = {'class' : 'product-image-link'})
    for tag_link in tag_links:
        product_link = tag_link['href'] + '\n'
        list_links.append(product_link)
        
print(len(list_links), list_links)
write_txt('tivi_links.txt', list_links)
        
        


KeyboardInterrupt: 

In [15]:
list_links = read_txt('tivi_links.txt')
list_names = []
list_prices = []
list_secondary_prices = []
list_images = []
for i in range(len(list_links)):
    url = list_links[i].rstrip('\n')
    print(url)
    page_response = requests.get(url)
    page_content = BeautifulSoup(page_response.content, 'html.parser')
    
    #ten san pham
    product_name = page_content.find(name = 'h1').text
    list_names.append(product_name)
    product_discount_price = page_content.find(name = 'div', attrs = {'class' : 'online-sale-price'}).find(name = 'span', attrs= {'class' : 'price-real color'}).text
    list_secondary_prices.append(product_discount_price)
    product_original_price = page_content.find(name = 'div', attrs = {'class' : 'online-sale-price'}).find(name = 'span',attrs= {'class' : 'price-sale'}).text
    list_prices.append(product_original_price)
    img_link = page_content.find(name = 'img', attrs = {'class' : 'bzoom_thumb_image'})['src']
    img = img_link.split('/')[-1]
    dict_tivi = {
        'name' : product_name,
        'price' : product_original_price,
        'discount_price' : product_discount_price,
        'image' : img
    }
    # if(i ==100):
    #     break
    write_json(f'json/{i}tivi.json', dict_tivi)
print(len(list_names), len(list_prices), len(list_secondary_prices))

https://dienmaythiennamhoa.vn/tivi/google-tivi-sony-4k-75-inch-k-75s30-258966.html
https://dienmaythiennamhoa.vn/tivi/google-tivi-sony-4k-65-inch-k-65s30-258965.html
https://dienmaythiennamhoa.vn/tivi/google-tivi-sony-4k-55-inch-k-55s30-258964.html
https://dienmaythiennamhoa.vn/tivi/google-tivi-sony-4k-50-inch-k-50s30-258963.html
https://dienmaythiennamhoa.vn/tivi/google-tivi-sony-4k-43-inch-k-43s30-258962.html
https://dienmaythiennamhoa.vn/tivi/google-tivi-sony-4k-85-inch-kd-85x80l-258949.html
https://dienmaythiennamhoa.vn/tivi/internet-tivi-casper-4k-58-inch-58ux5200-258930.html
https://dienmaythiennamhoa.vn/tivi/android-tivi-casper-qled-4k-55-inch-55qg8000-258929.html
https://dienmaythiennamhoa.vn/tivi/google-tivi-tcl-qled-4k-98-inch-98c655-258928.html
https://dienmaythiennamhoa.vn/tivi/google-tivi-tcl-qled-4k-85-inch-85c655-258927.html
https://dienmaythiennamhoa.vn/tivi/google-tivi-tcl-qled-4k-75-inch-75c655-258926.html
https://dienmaythiennamhoa.vn/tivi/google-tivi-tcl-qled-4k-65-

ConnectionError: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))

In [ ]:
df = pd.DataFrame({'ten': list_names, 'Gia goc': list_prices, 'Gia giam' : list_secondary_prices})
df.head(20)

,ten,Gia goc,Gia giam
0,Google tivi Sony 4K 75 inch K-75S30,39690000.00đ,32390000.00đ
1,Google tivi Sony 4K 65 inch K-65S30,26990000.00đ,21890000.00đ
2,Google tivi Sony 4K 55 inch K-55S30,22490000.00đ,17890000.00đ
3,Google tivi Sony 4K 50 inch K-50S30,18490000.00đ,15890000.00đ
4,Google tivi Sony 4K 43 inch K-43S30,17790000.00đ,13890000.00đ
5,Google tivi Sony 4K 85 inch KD-85X80L,53890000.00đ,39890000.00đ
6,Internet tivi Casper 4K 58 inch 58UX5200,13990000.00đ,6990000.00đ
7,Android tivi Casper QLED 4K 55 inch 55QG8000,11490000.00đ,7990000.00đ
8,Google tivi TCL QLED 4K 98 inch 98C655,70690000.00đ,69990000.00đ
9,Google tivi TCL QLED 4K 85 inch 85C655,40690000.00đ,39990000.00đ
